In [ ]:
!pip install matplotlib networkx torch gym numpy imageio seaborn procgen gym3 dataclasses 

In [ ]:
!pip install stable_baselines3 


In [3]:
#import statements
import matplotlib.patches as patches
import networkx as nx
import torch.distributions
import torch
from procgen_wrappers import VecExtractDictObs, TransposeFrame, ScaledFloatFrame
import gym
import random
import numpy as np
import helpers
from helpers import generate_action, load_model
import imageio
import matplotlib.pyplot as plt
import typing
import math

from procgen import ProcgenGym3Env
import struct
import typing
from typing import Tuple, Dict, Callable, List, Optional
from dataclasses import dataclass
from src.policies_impala import ImpalaCNN

from gym3 import ToBaselinesVecEnv
import seaborn as sns
import random

%load_ext autoreload
%autoreload 2

In [4]:
# Dictionary of ordered layer names
ordered_layer_names = {
 1: 'conv_seqs.0',
 2: 'conv_seqs.0.conv',
 3: 'conv_seqs.0.max_pool2d',
 4: 'conv_seqs.0.res_block0',
 5: 'conv_seqs.0.res_block0.conv0',
 6: 'conv_seqs.0.res_block0.conv1',
 7: 'conv_seqs.0.res_block1',
 8: 'conv_seqs.0.res_block1.conv0',
 9: 'conv_seqs.0.res_block1.conv1',
 10: 'conv_seqs.1',
 11: 'conv_seqs.1.conv',
 12: 'conv_seqs.1.max_pool2d',
 13: 'conv_seqs.1.res_block0',
 14: 'conv_seqs.1.res_block0.conv0',
 15: 'conv_seqs.1.res_block0.conv1',
 16: 'conv_seqs.1.res_block1',
 17: 'conv_seqs.1.res_block1.conv0',
 18: 'conv_seqs.1.res_block1.conv1',
 19: 'conv_seqs.2',
 20: 'conv_seqs.2.conv',
 21: 'conv_seqs.2.max_pool2d',
 22: 'conv_seqs.2.res_block0',
 23: 'conv_seqs.2.res_block0.conv0',
 24: 'conv_seqs.2.res_block0.conv1',
 25: 'conv_seqs.2.res_block1',
 26: 'conv_seqs.2.res_block1.conv0',
 27: 'conv_seqs.2.res_block1.conv1',
 28: 'hidden_fc',
 29: 'logits_fc',
 30: 'value_fc'
}

# Extracting all the names into a list
layer_names = list(ordered_layer_names.values())
channels = {}
for layer_name in layer_names:
    if layer_name.startswith("conv_seqs.0"):
        channels[layer_name] = 16
    elif layer_name.startswith("conv"):
        channels[layer_name] = 32
    else:
        channels[layer_name] = 1

print(channels)


{'conv_seqs.0': 16, 'conv_seqs.0.conv': 16, 'conv_seqs.0.max_pool2d': 16, 'conv_seqs.0.res_block0': 16, 'conv_seqs.0.res_block0.conv0': 16, 'conv_seqs.0.res_block0.conv1': 16, 'conv_seqs.0.res_block1': 16, 'conv_seqs.0.res_block1.conv0': 16, 'conv_seqs.0.res_block1.conv1': 16, 'conv_seqs.1': 32, 'conv_seqs.1.conv': 32, 'conv_seqs.1.max_pool2d': 32, 'conv_seqs.1.res_block0': 32, 'conv_seqs.1.res_block0.conv0': 32, 'conv_seqs.1.res_block0.conv1': 32, 'conv_seqs.1.res_block1': 32, 'conv_seqs.1.res_block1.conv0': 32, 'conv_seqs.1.res_block1.conv1': 32, 'conv_seqs.2': 32, 'conv_seqs.2.conv': 32, 'conv_seqs.2.max_pool2d': 32, 'conv_seqs.2.res_block0': 32, 'conv_seqs.2.res_block0.conv0': 32, 'conv_seqs.2.res_block0.conv1': 32, 'conv_seqs.2.res_block1': 32, 'conv_seqs.2.res_block1.conv0': 32, 'conv_seqs.2.res_block1.conv1': 32, 'hidden_fc': 1, 'logits_fc': 1, 'value_fc': 1}


In [5]:

def wrap_venv(venv) -> ToBaselinesVecEnv:
    "Wrap a vectorized env, making it compatible with the gym apis, transposing, scaling, etc."

    venv = ToBaselinesVecEnv(venv)  
    venv = VecExtractDictObs(venv, "rgb")

    venv = TransposeFrame(venv)
    venv = ScaledFloatFrame(venv)
    return venv  

def create_venv(
    num: int, start_level: int, num_levels: int, num_threads: int = 1, distribution_mode: str = "easy"
):
    """
    Create a wrapped venv. See https://github.com/openai/procgen#environment-options for params

    num=1 - The number of parallel environments to create in the vectorized env.

    num_levels=0 - The number of unique levels that can be generated. Set to 0 to use unlimited levels.

    start_level=0 - The lowest seed that will be used to generated levels. 'start_level' and 'num_levels' fully specify the set of possible levels.
    """
    venv = ProcgenGym3Env(
        num=num,
        env_name="heist",
        num_levels=num_levels,
        start_level=start_level,
        distribution_mode=distribution_mode,
        num_threads=num_threads,
        render_mode="rgb_array",
    )
    venv = wrap_venv(venv)
    return venv

In [6]:
from collections import defaultdict
from typing import List, Dict, Tuple
 
class Node:
    def __init__(self, name: str):
        self.name = name
        self.visited = False
        self.children = []
        self.parents = []


    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parents.append(self)

    def remove_child(self, child_node):
        if child_node in self.children:
            self.children.remove(child_node)
        if self in child_node.parents:
            child_node.parents.remove(self)

class Graph:
    def __init__(self):
        self.nodes = {}  # Maps node names to Node objects

    def add_node(self, name: str):
        if name not in self.nodes:
            self.nodes[name] = Node(name)
        return self.nodes[name]

    def add_edge(self, parent_name: str, child_name: str):
        parent_node = self.add_node(parent_name)
        child_node = self.add_node(child_name)
        parent_node.add_child(child_node)
    def add_edge_with_channels(self, parent_name: str, child_name: str):
        parent_channels = channels[parent_name]
        child_channels = channels[child_name]
        for parent_channel in range(parent_channels):
            for child_channel in range(child_channels):
                self.add_edge(parent_name+".channel"+str(parent_channel), child_name+".channel" + str(child_channel))

    def remove_edge(self, parent_name: str, child_name: str):
        try:
            parent_node = self.nodes[parent_name]
            child_node = self.nodes[child_name]
            parent_node.remove_child(child_node)
        except KeyError as e:
            print(f"Couldn't index in - are you sure this edge exists between {parent_name} and {child_name}?")
            raise e

    def display(self):
        for name, node in self.nodes.items():
            children_names = [child.name for child in node.children]
            parent_names = [parent.name for parent in node.parents]
            print(f'Node {name} has children: {children_names} and parents: {parent_names}')
def build_graph():
    graph = Graph()

    # Define the sequence names and their respective channels
    sequences = ['conv_seqs.0', 'conv_seqs.1', 'conv_seqs.2']
    last_seq_output = 'input'

    # Connect each sequence and define internal module connections
    for i, seq in enumerate(sequences):
        #add the skip connections
        graph.add_edge_with_channels(f'{seq}.max_pool2d', f'{seq}.res_block0')
        graph.add_edge_with_channels(f'{seq}.res_block0', f'{seq}.res_block1')  

        # Connect within ConvSequence
        graph.add_edge_with_channels(f'{seq}.conv', f'{seq}.max_pool2d')
        graph.add_edge_with_channels(f'{seq}.max_pool2d', f'{seq}.res_block0.conv0')
        graph.add_edge_with_channels(f'{seq}.res_block0.conv0', f'{seq}.res_block0.conv1')
        graph.add_edge_with_channels(f'{seq}.res_block0.conv1', f'{seq}.res_block0')

        graph.add_edge_with_channels(f'{seq}.res_block0', f'{seq}.res_block1.conv0')
        graph.add_edge_with_channels(f'{seq}.res_block1.conv0', f'{seq}.res_block1.conv1')
        graph.add_edge_with_channels(f'{seq}.res_block1.conv1', f'{seq}.res_block1')    
        graph.add_edge_with_channels(f'{seq}.res_block1',f'{seq}')

    # Connect hidden_fc to logits_fc and value_fc
    graph.add_edge_with_channels(f'{sequences[0]}', f'{sequences[1]}.conv')
    graph.add_edge_with_channels(f'{sequences[1]}', f'{sequences[2]}.conv')

    for i in range(32):
        graph.add_edge(f'{sequences[2]}.channel{i}', 'hidden_fc')

    graph.add_node('hidden_fc')
    graph.add_node('logits_fc')
    graph.add_node('value_fc')
    graph.add_edge('hidden_fc', 'logits_fc')
    graph.add_edge('hidden_fc', 'value_fc')

    return graph

In [7]:
model_path = "../model_final.pt"
model = helpers.load_model(model_path=model_path)
graph = build_graph()
graph.display()

Node conv_seqs.0.max_pool2d.channel0 has children: ['conv_seqs.0.res_block0.channel0', 'conv_seqs.0.res_block0.channel1', 'conv_seqs.0.res_block0.channel2', 'conv_seqs.0.res_block0.channel3', 'conv_seqs.0.res_block0.channel4', 'conv_seqs.0.res_block0.channel5', 'conv_seqs.0.res_block0.channel6', 'conv_seqs.0.res_block0.channel7', 'conv_seqs.0.res_block0.channel8', 'conv_seqs.0.res_block0.channel9', 'conv_seqs.0.res_block0.channel10', 'conv_seqs.0.res_block0.channel11', 'conv_seqs.0.res_block0.channel12', 'conv_seqs.0.res_block0.channel13', 'conv_seqs.0.res_block0.channel14', 'conv_seqs.0.res_block0.channel15', 'conv_seqs.0.res_block0.conv0.channel0', 'conv_seqs.0.res_block0.conv0.channel1', 'conv_seqs.0.res_block0.conv0.channel2', 'conv_seqs.0.res_block0.conv0.channel3', 'conv_seqs.0.res_block0.conv0.channel4', 'conv_seqs.0.res_block0.conv0.channel5', 'conv_seqs.0.res_block0.conv0.channel6', 'conv_seqs.0.res_block0.conv0.channel7', 'conv_seqs.0.res_block0.conv0.channel8', 'conv_seqs.0.

In [20]:
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

def format_layer_name(layer_name):
    return layer_name.replace('.', '_')

class ModelActivations:
    def __init__(self, model):
        self.activations = {}
        self.model = model
        self.hooks = []  # To keep track of hooks
        self.layer_paths = layer_names

    def clear_hooks(self):
        # Remove all previously registered hooks
        for hook in self.hooks:
            hook.remove()
        self.hooks = []
        self.activations = {}

    def get_activation(self, name):
        def hook(model, input, output):
            processed_output = []
            for item in output:
                if isinstance(item, torch.Tensor):
                    processed_output.append(item.detach())
                elif isinstance(item, torch.distributions.Categorical):
                    processed_output.append(item.logits.detach())
                else:
                    processed_output.append(item)
            self.activations[name] = tuple(processed_output)
        return hook

    def register_hook_by_path(self, path, name):
        elements = path.split('.')
        model = self.model
        for i, element in enumerate(elements):
            if '[' in element:
                base, index = element.replace(']', '').split('[')
                index = int(index)
                model = getattr(model, base)[index]
            else:
                model = getattr(model, element)
            if i == len(elements) - 1:
                hook = model.register_forward_hook(self.get_activation(name))
                self.hooks.append(hook)  # Keep track of the hook

    def run_with_cache(self, input):
        self.clear_hooks()  # Clear any existing hooks
        self.activations = {}  # Reset activations

        # Handle edge case: input is not a tensor
        if not isinstance(input, torch.Tensor):
            input = torch.tensor(input, dtype=torch.float32)


        # Check the shape of the input and reshape if necessary
        if input.shape == torch.Size([1, 3, 64, 64]):
            input = input.squeeze(0)  # Remove the batch dimension
        if input.shape == torch.Size([3, 64, 64]):
            input = input.permute(1, 2, 0)  # Switch dimensions to (64, 64, 3)


        # Handle edge case: empty layer_paths
        if not self.layer_paths:
            output = self.model(input)
            return output, self.activations

        # Register hooks for each layer path
        for path in self.layer_paths:
            try:
                self.register_hook_by_path(path, path.replace('.', '_'))
            except AttributeError:
                print(f"Warning: Layer '{path}' not found in the model. Skipping hook registration.")

        # Add batch dimension if missing
        if input.dim() == 3:
            input = input.unsqueeze(0)

        # Run the model with the registered hooks
        output = self.model(input)

        return output, self.activations
    
    def patch_activations(self, input, corrupt_cache= {},to_change_layer_name=[],ablate =False):
        #print("to change layer name", to_change_layer_name)
        layer_name_channels = {}
        for name in to_change_layer_name:
            if "_channel" in name:
                if name.split("_channel")[0] in layer_name_channels:
                    layer_name_channels[name.split("_channel")[0]].append(int(name.split("_channel")[1]))
                else:
                    layer_name_channels[name.split("_channel")[0]] = [int(name.split("_channel")[1])]
        #print("to change layer names and layer name channels", to_change_layer_name, layer_name_channels.keys())

        cached_activations = {}

        # Define a factory function to create hook functions with a stable layer name
        def make_hook(layer_name):
            def saves_cache(module, input, output):
                cached_activations[layer_name] = output.detach()
                formatted_layer_name = format_layer_name(layer_name)        
 
                if formatted_layer_name in layer_name_channels:
     
                    if ablate:
                        for channel in layer_name_channels[formatted_layer_name]:
                            output[:,channel,:,:]  = 0
                        return output
                    else:
                        to_patch_activation_tensor = corrupt_cache[formatted_layer_name][0].unsqueeze(0)
                        if to_patch_activation_tensor.shape == output.shape:  
                            #print("patching an output with channels", output.shape)
                            for channel in layer_name_channels[formatted_layer_name]:
                                output[:,channel,:,:]  = to_patch_activation_tensor[:,channel,:,:]
                            return output
                        else:
                            print("incompatible shapes")
                elif formatted_layer_name in to_change_layer_name:
                    to_patch_activation_tensor = corrupt_cache[formatted_layer_name][0].unsqueeze(0)
                    if to_patch_activation_tensor.shape == output.shape:       
                            
                        return to_patch_activation_tensor

            return saves_cache
        
        # Function to recursively get a sub-module from its name
        def get_submodule(module, submodule_name):
            names = submodule_name.split('.')
            for name in names:
                module = getattr(module, name)
            return module
        
        for name in layer_names:
            layer = get_submodule(model, name)
            hook = make_hook(name)  # Create a hook with a stable layer name
            self.hooks.append(layer.register_forward_hook(hook))


        # Handle edge case: input is not a tensor
        if not isinstance(input, torch.Tensor):
            input = torch.tensor(input, dtype=torch.float32)


        # Check the shape of the input and reshape if necessary
        if input.shape == torch.Size([1, 3, 64, 64]):
            input = input.squeeze(0)  # Remove the batch dimension
        if input.shape == torch.Size([3, 64, 64]):
            input = input.permute(1, 2, 0)  # Switch dimensions to (64, 64, 3)
        if input.dim() == 3:
                input = input.unsqueeze(0)
        output = self.model(input)

        self.clear_hooks()

        return output,cached_activations

In [21]:
class RLPolicyExperiment():
    """
    A class to manage an experiment that adjusts and analyzes the components of an RL policy
    network dynamically during training, akin to the TLACDCExperiment for neural networks.
    """

    def __init__(self, model, threshold,graph,input,corrupt_input, ablate):
        self.model = model
        self.model_activations = ModelActivations(model)
        #this should be input dataset
        self.input = input
        self.corrupt_input = corrupt_input
        self.threshold = threshold
        # Node definitions are based on the provided structure
        self.graph = graph
        self.removed_nodes = []
        self.removed_edges = []
        self.ablate = ablate
        self.mean_kl_divergences = 0

        

    def reverse_topologically_sort_corr(self):
        #sorts the graph in reverse topological order
        order = []
        # Set all nodes as unvisited
        for node in self.graph.nodes.values():
            node.visited = False

        # Helper function to perform DFS
        def dfs(node):
            node.visited = True
            for child in node.children:
                if not child.visited:
                    dfs(child)
            # Prepend node to maintain reverse order
            order.insert(0, node.name)

        # Perform DFS from each unvisited node
        for node in self.graph.nodes.values():
            if not node.visited:
                dfs(node)
        return order

            
    def _on_step(self):
        kl_divergences = []
        clean_logits, _ = self.model_activations.run_with_cache(self.input) # Get original logits with normal activations
        self.model_activations.clear_hooks()
        
        _, corrupt_cache = self.model_activations.run_with_cache(self.corrupt_input) # Get original logits with normal activations
        self.model_activations.clear_hooks()
        
        
        order = self.reverse_topologically_sort_corr()
        order.reverse()


        for node in order:

            current_node = self.graph.nodes[node]
            print("here is the current node", current_node)
            
            # Simulate zero activation for the first node and get the kl divergence
            for parent_node in current_node.parents:
                to_patch_nodes = self.removed_nodes.copy()
                if parent_node.name not in to_patch_nodes:
                    to_patch_nodes.append(format_layer_name(parent_node.name))
  
                if self.ablate:
                    modified_logits, _ = self.model_activations.patch_activations(self.input, corrupt_cache =corrupt_cache,to_change_layer_name=to_patch_nodes,ablate =True)
                else:
                    modified_logits, _ = self.model_activations.patch_activations(input =self.input, corrupt_cache=corrupt_cache,to_change_layer_name=to_patch_nodes, ablate= False)
                self.model_activations.clear_hooks()
                
                prob1 = torch.softmax(clean_logits[0].logits, dim=1)
                prob2 = torch.softmax(modified_logits[0].logits, dim=1)
                kl_div = torch.nn.functional.kl_div(torch.log(prob1), prob2, reduction='batchmean')
                print("the kl divergence is", kl_div)
                
                kl_divergences.append(kl_div)
                

                remove = kl_div < self.threshold 

                if remove:
                    print("permanently removing an edge in the graph", parent_node.name, current_node.name)
                    self.graph.remove_edge(parent_node.name,current_node.name)
                    self.removed_edges.append((parent_node,current_node))
                    if parent_node.name not in self.removed_nodes:
                        self.removed_nodes.append(format_layer_name(parent_node.name))
        self.mean_kl_divergences =sum(kl_divergences) / len(kl_divergences)
       
      

In [ ]:
import heist
start_level = random.randint(1, 10000)
venv = create_venv(num=1, num_levels=1, start_level=start_level)
state = heist.state_from_venv(venv, 0)
corrupt_input = venv.reset()    #the input with the blue key is the corrupt input
corrupt_input = helpers.observation_to_rgb(corrupt_input)
state_values = state.state_vals

for ents in state_values["ents"]:
    if ents["image_type"].val== 9:
        gem_x = ents["x"].val 
        gem_y = ents["y"].val 

state.remove_gem()

state_bytes = state.state_bytes
if state_bytes is not None:
    venv.env.callmethod("set_state", [state_bytes])
    input = venv.reset()
    input = helpers.observation_to_rgb(input)


experiment = RLPolicyExperiment(model,0.00001,build_graph(),input,corrupt_input,ablate=False)
experiment._on_step()
#print(experiment.graph.display())
print("the mean of the kl divergences are" , experiment.mean_kl_divergences)


here is the current node <__main__.Node object at 0x7faffbfc1f60>
the kl divergence is tensor(0.0003, grad_fn=<DivBackward0>)
here is the current node <__main__.Node object at 0x7faffbfc3790>
the kl divergence is tensor(0.0003, grad_fn=<DivBackward0>)
here is the current node <__main__.Node object at 0x7fb0002a4880>
the kl divergence is tensor(0., grad_fn=<DivBackward0>)
permanently removing an edge in the graph conv_seqs.2.channel0 hidden_fc
the kl divergence is tensor(0., grad_fn=<DivBackward0>)
permanently removing an edge in the graph conv_seqs.2.channel2 hidden_fc
the kl divergence is tensor(0., grad_fn=<DivBackward0>)
permanently removing an edge in the graph conv_seqs.2.channel4 hidden_fc
the kl divergence is tensor(0., grad_fn=<DivBackward0>)
permanently removing an edge in the graph conv_seqs.2.channel6 hidden_fc
the kl divergence is tensor(0., grad_fn=<DivBackward0>)
permanently removing an edge in the graph conv_seqs.2.channel8 hidden_fc
the kl divergence is tensor(0., grad_